# DuckDB 全文检索扩展

Full-Text Search 是 DuckDB 的一个扩展，允许对字符串进行搜索，类似于 SQLite 的 FTS5 扩展。

## 安装和加载

`fts` 扩展会从官方扩展库中在首次使用时透明地自动加载。如果想要手动安装和加载，可以运行相应的命令。

## 使用方法

该扩展向 DuckDB 添加了两个 `PRAGMA` 语句：一个用于创建索引，一个用于删除索引。此外，还添加了一个标量宏 `stem`，它在扩展内部使用。

### `PRAGMA create_fts_index`



-- 创建全文检索索引的PRAGMA语句
create_fts_index(input_table, input_id, *input_values, stemmer = 'porter',
                 stopwords = 'english', ignore = '(\\.|[^a-z])+',
                 strip_accents = 1, lower = 1, overwrite = 0)

这是一个创建指定表的 FTS 索引的 `PRAGMA`。

| 参数 | 类型 | 描述 |
|------|------|------|
| `input_table` | `VARCHAR` | 指定表的限定名称，例如 `'table_name'` 或 `'main.table_name'` |
| `input_id` | `VARCHAR` | 文档标识符的列名，例如 `'document_identifier'` |
| `input_values...` | `VARCHAR` | 要索引的文本字段的列名（可变参数），例如 `'text_field_1'`, `'text_field_2'`, …, `'text_field_N'`，或者用 `'*'` 表示输入表中所有类型为 `VARCHAR` 的列 |
| `stemmer` | `VARCHAR` | 要使用的词干提取器类型。可选值：`'arabic'`, `'basque'`, `'catalan'`, `'danish'`, `'dutch'`, `'english'`, `'finnish'`, `'french'`, `'german'`, `'greek'`, `'hindi'`, `'hungarian'`, `'indonesian'`, `'irish'`, `'italian'`, `'lithuanian'`, `'nepali'`, `'norwegian'`, `'porter'`, `'portuguese'`, `'romanian'`, `'russian'`, `'serbian'`, `'spanish'`, `'swedish'`, `'tamil'`, `'turkish'`，或者 `'none'`（如果不想使用词干提取）。默认为 `'porter'` |
| `stopwords` | `VARCHAR` | 包含所需停用词的单列 `VARCHAR` 表的限定名称，或者 `'none'`（如果不使用停用词）。默认为 `'english'`，即预定义的 571 个英语停用词列表 |
| `ignore` | `VARCHAR` | 要忽略的模式的正则表达式。默认为 `'(\\.|[^a-z])+'`，忽略所有转义的和非小写英文字母的字符 |
| `strip_accents` | `BOOLEAN` | 是否去除重音（例如，将 `á` 转换为 `a`）。默认为 `1` |
| `lower` | `BOOLEAN` | 是否将所有文本转换为小写。默认为 `1` |
| `overwrite` | `BOOLEAN` | 是否覆盖表上已有的索引。默认为 `0` |

该 `PRAGMA` 在新创建的模式下构建索引。模式将根据输入表命名：如果在表 `'main.table_name'` 上创建索引，则模式将命名为 `'fts_main_table_name'`。

### `PRAGMA drop_fts_index`



-- 删除全文检索索引的PRAGMA语句
drop_fts_index(input_table)

删除指定表的 FTS 索引。

| 参数 | 类型 | 描述 |
|------|------|------|
| `input_table` | `VARCHAR` | 输入表的限定名称，例如 `'table_name'` 或 `'main.table_name'` |

### `match_bm25` 函数



-- BM25检索函数，用于在索引中搜索文档
match_bm25(input_id, query_string, fields := NULL, k := 1.2, b := 0.75, conjunctive := 0)

当构建索引时，会创建这个检索宏，可用于搜索索引。

| 参数 | 类型 | 描述 |
|------|------|------|
| `input_id` | `VARCHAR` | 文档标识符的列名，例如 `'document_identifier'` |
| `query_string` | `VARCHAR` | 要在索引中搜索的字符串 |
| `fields` | `VARCHAR` | 要搜索的字段的逗号分隔列表，例如 `'text_field_2, text_field_N'`。默认为 `NULL`，表示搜索所有已索引的字段 |
| `k` | `DOUBLE` | Okapi BM25 检索模型中的参数 _k1_。默认为 `1.2` |
| `b` | `DOUBLE` | Okapi BM25 检索模型中的参数 _b_。默认为 `0.75` |
| `conjunctive` | `BOOLEAN` | 是否使查询为合取的，即查询字符串中的所有术语都必须存在，文档才会被检索到 |

### `stem` 函数



-- 词干提取函数，将单词转换为其基本形式
stem(input_string, stemmer)

将单词缩减为基本形式。在扩展内部使用。

| 参数 | 类型 | 描述 |
|------|------|------|
| `input_string` | `VARCHAR` | 要进行词干提取的列或常量 |
| `stemmer` | `VARCHAR` | 要使用的词干提取器类型。可以是 `'arabic'`, `'basque'`, 等多种语言类型，或 `'none'` |

## 使用示例

### 创建测试数据

创建一个表并用文本数据填充：



-- 创建文档表，包含文档ID、内容、作者和版本信息
CREATE TABLE documents (
    document_identifier VARCHAR,  -- 文档唯一标识符
    text_content VARCHAR,         -- 文档文本内容
    author VARCHAR,               -- 文档作者
    doc_version INTEGER           -- 文档版本号
);

-- 插入两条示例文档数据
INSERT INTO documents
    VALUES ('doc1',
            'The mallard is a dabbling duck that breeds throughout the temperate.',
            'Hannes Mühleisen',
            3),
           ('doc2',
            'The cat is a domestic species of small carnivorous mammal.',
            'Laurens Kuiper',
            2
           );

**Python实现**：



In [50]:
import duckdb

# 创建DuckDB内存数据库连接
conn = duckdb.connect(':memory:')

# 创建文档表，包含文档ID、内容、作者和版本字段
conn.execute("""
CREATE TABLE documents (
    document_identifier VARCHAR,  -- 文档唯一标识符
    text_content VARCHAR,         -- 文档文本内容
    author VARCHAR,               -- 文档作者
    doc_version INTEGER           -- 文档版本号
)
""")

# 插入两条示例文档数据
conn.execute("""
INSERT INTO documents
    VALUES ('doc1',
            'The mallard is a dabbling duck that breeds throughout the temperate.',
            'Hannes Mühleisen',
            3),
           ('doc2',
            'The cat is a domestic species of small carnivorous mammal.',
            'Laurens Kuiper',
            2
           )
""")

# 验证数据是否成功插入，并打印结果
result = conn.execute("SELECT * FROM documents").fetchall()
print("已插入的文档数据:")
for row in result:
    print(row)

已插入的文档数据:
('doc1', 'The mallard is a dabbling duck that breeds throughout the temperate.', 'Hannes Mühleisen', 3)
('doc2', 'The cat is a domestic species of small carnivorous mammal.', 'Laurens Kuiper', 2)


### 构建索引

构建索引，并使 `text_content` 和 `author` 列可搜索。



-- 为documents表创建全文检索索引，索引document_identifier列，并对text_content和author列建立搜索索引
PRAGMA create_fts_index(
    'documents', 'document_identifier', 'text_content', 'author'
);

**Python实现**：



In [51]:
# 安装并加载全文检索(FTS)扩展
conn.execute("INSTALL fts")  # 安装FTS扩展
conn.execute("LOAD fts")     # 加载FTS扩展到当前连接

# 为documents表创建全文检索索引
# 参数说明: 表名、ID列名、要索引的文本列(这里同时索引text_content和author两列)
conn.execute("""
PRAGMA create_fts_index(
    'documents', 'document_identifier', 'text_content', 'author'
)
""")
print("全文检索索引已创建")

全文检索索引已创建


### 搜索示例

在 `author` 字段索引中搜索由 `Muhleisen` 撰写的文档。这会检索到 `doc1` ：



-- 在author字段中搜索包含"Muhleisen"的文档
-- 并且仅返回doc_version大于2的文档
-- 按相关性得分降序排列结果
SELECT document_identifier, text_content, score
FROM (
    SELECT *, fts_main_documents.match_bm25(
        document_identifier,  -- 文档标识符列
        'Muhleisen',         -- 搜索关键词
        fields := 'author'   -- 限定只在author字段中搜索
    ) AS score
    FROM documents
) sq
WHERE score IS NOT NULL      -- 仅保留匹配的文档(有分数的)
  AND doc_version > 2        -- 文档版本必须大于2
ORDER BY score DESC;

**Python实现**：



In [52]:
# 按作者搜索，在author字段中查找"Muhleisen"
query = """
SELECT document_identifier, text_content, score
FROM (
    SELECT *, fts_main_documents.match_bm25(
        document_identifier,  -- 文档标识符列
        'Muhleisen',         -- 搜索关键词
        fields := 'author'   -- 限定只在author字段中搜索
    ) AS score
    FROM documents
) sq
WHERE score IS NOT NULL      -- 仅保留匹配的文档(有分数的)
  AND doc_version > 2        -- 文档版本必须大于2
ORDER BY score DESC          -- 按相关性得分降序排列
"""

# 执行搜索查询并获取结果
result = conn.execute(query).fetchall()
print("\n按作者 'Muhleisen' 搜索结果:")
for row in result:
    print(f"文档ID: {row[0]}, 内容: {row[1]}, 得分: {row[2]}")


按作者 'Muhleisen' 搜索结果:
文档ID: doc1, 内容: The mallard is a dabbling duck that breeds throughout the temperate., 得分: 0.3094700890003546


搜索关于 `small cats` 的文档。这会检索到 `doc2` ：



-- 搜索包含"small cats"相关内容的文档
-- 在所有已索引的字段(text_content和author)中搜索
-- 按相关性得分降序排列结果
SELECT document_identifier, text_content, score
FROM (
    SELECT *, fts_main_documents.match_bm25(
        document_identifier,  -- 文档标识符列
        'small cats'         -- 搜索关键词
    ) AS score               -- 未指定fields参数，将在所有索引字段中搜索
    FROM documents
) sq
WHERE score IS NOT NULL      -- 仅保留匹配的文档(有分数的)
ORDER BY score DESC;

**Python实现**：



In [53]:
# 按内容关键词搜索，查找包含"small cats"的文档
query = """
SELECT document_identifier, text_content, score
FROM (
    SELECT *, fts_main_documents.match_bm25(
        document_identifier,  -- 文档标识符列
        'small cats'         -- 搜索关键词
    ) AS score               -- 未指定fields参数，将在所有索引字段中搜索
    FROM documents
) sq
WHERE score IS NOT NULL      -- 仅保留匹配的文档(有分数的)
ORDER BY score DESC          -- 按相关性得分降序排列
"""

# 执行搜索查询并获取结果
result = conn.execute(query).fetchall()
print("\n搜索 'small cats' 的结果:")
for row in result:
    print(f"文档ID: {row[0]}, 内容: {row[1]}, 得分: {row[2]}")

# 关闭数据库连接
conn.close()


搜索 'small cats' 的结果:
文档ID: doc2, 内容: The cat is a domestic species of small carnivorous mammal., 得分: 0.5860760977528838


> **警告**
>
> 当输入表发生变化时，FTS 索引不会自动更新。
> 可以通过重新创建索引来解决此限制。

## 完整Python示例代码

以下是一个完整的Python脚本，包含所有上述操作：



In [54]:
import duckdb

def run_duckdb_fts_demo():
    # 创建内存数据库连接
    conn = duckdb.connect(':memory:')
    
    try:
        # 步骤1: 安装并加载全文检索扩展
        conn.execute("INSTALL fts")  # 安装FTS扩展包
        conn.execute("LOAD fts")     # 加载FTS扩展到当前会话
        print("FTS扩展已加载")
        
        # 步骤2: 创建文档数据表
        conn.execute("""
        CREATE TABLE documents (
            document_identifier VARCHAR,  -- 文档唯一标识符
            text_content VARCHAR,         -- 文档文本内容
            author VARCHAR,               -- 文档作者
            doc_version INTEGER           -- 文档版本号
        )
        """)
        
        # 步骤3: 插入示例数据
        conn.execute("""
        INSERT INTO documents
            VALUES ('doc1',
                    'The mallard is a dabbling duck that breeds throughout the temperate.',
                    'Hannes Mühleisen',
                    3),
                   ('doc2',
                    'The cat is a domestic species of small carnivorous mammal.',
                    'Laurens Kuiper',
                    2
                   )
        """)
        
        print("文档表已创建并填充数据")
        
        # 步骤4: 创建全文检索索引
        # 为documents表创建索引，以document_identifier为ID列，索引text_content和author字段
        conn.execute("""
        PRAGMA create_fts_index(
            'documents', 'document_identifier', 'text_content', 'author'
        )
        """)
        print("全文检索索引已创建")
        
        # 步骤5: 按作者搜索示例
        # 在author字段中搜索'Muhleisen'，并限制只返回版本大于2的文档
        author_query = """
        SELECT document_identifier, text_content, score
        FROM (
            SELECT *, fts_main_documents.match_bm25(
                document_identifier,
                'Muhleisen',
                fields := 'author'
            ) AS score
            FROM documents
        ) sq
        WHERE score IS NOT NULL
          AND doc_version > 2
        ORDER BY score DESC
        """
        
        # 执行作者搜索查询并显示结果
        author_results = conn.execute(author_query).fetchall()
        print("\n按作者 'Muhleisen' 搜索结果:")
        for row in author_results:
            print(f"文档ID: {row[0]}, 内容: {row[1]}, 得分: {row[2]}")
        
        # 步骤6: 按内容关键词搜索示例
        # 在所有索引字段中搜索包含'small cats'的文档
        content_query = """
        SELECT document_identifier, text_content, score
        FROM (
            SELECT *, fts_main_documents.match_bm25(
                document_identifier,
                'small cats'
            ) AS score
            FROM documents
        ) sq
        WHERE score IS NOT NULL
        ORDER BY score DESC
        """
        
        # 执行内容搜索查询并显示结果
        content_results = conn.execute(content_query).fetchall()
        print("\n搜索 'small cats' 的结果:")
        for row in content_results:
            print(f"文档ID: {row[0]}, 内容: {row[1]}, 得分: {row[2]}")
            
    finally:
        # 步骤7: 清理资源，关闭数据库连接
        conn.close()
        print("\nDuckDB连接已关闭")

if __name__ == "__main__":
    run_duckdb_fts_demo()  # 执行演示程序

FTS扩展已加载
文档表已创建并填充数据
全文检索索引已创建

按作者 'Muhleisen' 搜索结果:
文档ID: doc1, 内容: The mallard is a dabbling duck that breeds throughout the temperate., 得分: 0.3094700890003546

搜索 'small cats' 的结果:
文档ID: doc2, 内容: The cat is a domestic species of small carnivorous mammal., 得分: 0.5860760977528838

DuckDB连接已关闭


## DuckDB中文全文检索实现

DuckDB自带的全文检索(FTS)扩展不直接支持中文分词，因此无法有效索引和搜索中文内容。但我们可以结合jieba等中文分词工具，对中文文本进行预处理，使其适用于DuckDB的FTS索引。

### 基本思路

1. 使用jieba对中文文本进行分词
2. 将分词结果用空格连接，转换为类似英文的形式
3. 对处理后的文本创建FTS索引
4. 查询时，对查询关键词也进行分词处理
5. 使用处理后的关键词在索引中搜索

### 实现方案

#### 第一步：安装必要的库



pip install duckdb jieba

#### 第二步：创建中文文本预处理函数



In [55]:
import jieba
import re

# 定义中文停用词列表 (可根据实际需求扩展或使用更完善的停用词库)
# 这里仅为示例，实际应用中建议使用更全面的停用词表
CHINESE_STOPWORDS = {
    "的", "了", "是", "在", "也", "和", "就", "都", "而", "及", "与", "著", "啊",
    "嗯", "哦", "哈", "嘿", "喂", "哎", "呢", "吧", "吗", "啦", "嘛", "哇",
    "我", "你", "他", "她", "它", "我们", "你们", "他们", "她们", "它们",
    "这", "那", "这些", "那些", "这个", "那个",
    "一", "一些", "一种", "一下", "一个", "一切", "一旦",
    "不", "没", "有", "无", "很", "太", "更", "最",
    "会", "能", "可", "要", "想", "得", "应", "该",
    "通过", "根据", "由于", "为了", "因为", "所以",
    "如果", "那么", "但是", "然而",
    "对", "从", "向", "于", "以", "之", "其", "或", "或者",
    "等", "等等", "例如", "比如", "其他", "其它", "另外", "还有", "以及"
}

def preprocess_chinese_text(text):
    """
    对中文文本进行预处理，包括分词、去除停用词和格式化
    
    Args:
        text (str): 原始中文文本
        
    Returns:
        str: 分词并去除停用词后的文本，词语间以空格分隔
    """
    if not text or not isinstance(text, str):
        return ""
    
    # 使用jieba进行分词
    words = jieba.cut(text)
    
    # 过滤掉停用词、标点符号、单个字符（可选，视情况而定）等
    filtered_words = [word for word in words 
                     if word.strip() and 
                     not re.match(r'[^\w\u4e00-\u9fff]+', word) and 
                     word not in CHINESE_STOPWORDS and
                     len(word) > 0]  # 确保词不为空且不在停用词表中
    
    # 将分词结果用空格连接
    return " ".join(filtered_words)

#### 第三步：创建数据表并进行文本预处理



In [56]:
import duckdb

# 创建数据库连接
conn = duckdb.connect(':memory:')

# 安装和加载FTS扩展
conn.execute("INSTALL fts")
conn.execute("LOAD fts")

# 创建原始中文文档表
conn.execute("""
CREATE TABLE chinese_documents (
    doc_id VARCHAR,       -- 文档ID
    title VARCHAR,        -- 文档标题
    content VARCHAR       -- 文档内容(中文)
)
""")

# 插入中文示例数据
conn.execute("""
INSERT INTO chinese_documents VALUES
    ('doc1', '人工智能简介', '人工智能是计算机科学的一个分支，致力于开发能够模拟人类智能的系统。'),
    ('doc2', '机器学习基础', '机器学习是人工智能的一个子领域，专注于让计算机从数据中学习。'),
    ('doc3', '深度学习技术', '深度学习是机器学习的一种方法，使用多层神经网络进行复杂模式识别。')
""")

# 创建预处理后的中文文档表
conn.execute("""
CREATE TABLE preprocessed_chinese_docs (
    doc_id VARCHAR,           -- 文档ID
    title_processed VARCHAR,  -- 经过分词处理的标题
    content_processed VARCHAR -- 经过分词处理的内容
)
""")

# 从Python中获取原始数据并进行预处理
documents = conn.execute("SELECT * FROM chinese_documents").fetchall()
for doc in documents:
    doc_id, title, content = doc
    # 对标题和内容进行分词处理
    title_processed = preprocess_chinese_text(title)
    content_processed = preprocess_chinese_text(content)
    
    # 将处理后的文本插入到新表中
    conn.execute("""
    INSERT INTO preprocessed_chinese_docs VALUES (?, ?, ?)
    """, (doc_id, title_processed, content_processed))

# 为预处理后的中文文档创建FTS索引
conn.execute("""
PRAGMA create_fts_index(
    'preprocessed_chinese_docs', 'doc_id', 'title_processed', 'content_processed'
)
""")

#### 第四步：执行中文搜索



In [57]:
def search_chinese(conn, query_text):
    """
    执行中文搜索
    
    Args:
        conn: DuckDB连接
        query_text (str): 中文查询文本
        
    Returns:
        list: 搜索结果列表
    """
    # 对查询文本进行分词
    processed_query = preprocess_chinese_text(query_text)
    
    # 执行搜索
    results = conn.execute("""
    SELECT sq.doc_id, cd.title, cd.content, score
    FROM (
        SELECT *, fts_main_preprocessed_chinese_docs.match_bm25(
            doc_id, 
            ?,  -- 使用处理后的查询字符串
            fields := 'title_processed, content_processed'
        ) AS score
        FROM preprocessed_chinese_docs
    ) sq
    JOIN chinese_documents cd ON sq.doc_id = cd.doc_id
    WHERE score IS NOT NULL
    ORDER BY score DESC
    """, (processed_query,)).fetchall()
    
    return results

# 执行搜索示例
search_results = search_chinese(conn, "机器学习和人工智能")
for result in search_results:
    doc_id, title, content, score = result
    print(f"文档ID: {doc_id}, 标题: {title}")
    print(f"内容: {content[:50]}...")
    print(f"相关性得分: {score}\n")

文档ID: doc2, 标题: 机器学习基础
内容: 机器学习是人工智能的一个子领域，专注于让计算机从数据中学习。...
相关性得分: 0.4000569815973582

文档ID: doc1, 标题: 人工智能简介
内容: 人工智能是计算机科学的一个分支，致力于开发能够模拟人类智能的系统。...
相关性得分: 0.21282652220049034

文档ID: doc3, 标题: 深度学习技术
内容: 深度学习是机器学习的一种方法，使用多层神经网络进行复杂模式识别。...
相关性得分: 0.2000284907986791



### 完整示例

下面是一个完整的中文全文检索演示程序：



In [58]:
import duckdb
import jieba
import re

# 定义中文停用词列表 (可根据实际需求扩展或使用更完善的停用词库)
# 这里仅为示例，实际应用中建议使用更全面的停用词表
CHINESE_STOPWORDS = {
    "的", "了", "是", "在", "也", "和", "就", "都", "而", "及", "与", "著", "啊",
    "嗯", "哦", "哈", "嘿", "喂", "哎", "呢", "吧", "吗", "啦", "嘛", "哇",
    "我", "你", "他", "她", "它", "我们", "你们", "他们", "她们", "它们",
    "这", "那", "这些", "那些", "这个", "那个",
    "一", "一些", "一种", "一下", "一个", "一切", "一旦",
    "不", "没", "有", "无", "很", "太", "更", "最",
    "会", "能", "可", "要", "想", "得", "应", "该",
    "通过", "根据", "由于", "为了", "因为", "所以",
    "如果", "那么", "但是", "然而",
    "对", "从", "向", "于", "以", "之", "其", "或", "或者",
    "等", "等等", "例如", "比如", "其他", "其它", "另外", "还有", "以及"
}

def preprocess_chinese_text(text):
    """对中文文本进行分词和去除停用词处理"""
    if not text or not isinstance(text, str):
        return ""
    words = jieba.cut(text)
    # 过滤掉停用词、标点符号、单个字符（可选）等
    filtered_words = [word for word in words 
                     if word.strip() and 
                     not re.match(r'[^\w\u4e00-\u9fff]+', word) and
                     word not in CHINESE_STOPWORDS and
                     len(word) > 0] # 确保词不为空且不在停用词表中
    return " ".join(filtered_words)

def run_chinese_fts_demo():
    # 创建内存数据库连接
    conn = duckdb.connect(':memory:')
    
    try:
        # 安装和加载FTS扩展
        conn.execute("INSTALL fts")
        conn.execute("LOAD fts")
        print("FTS扩展已加载")
        
        # 创建原始中文文档表
        conn.execute("""
        CREATE TABLE chinese_documents (
            doc_id VARCHAR,
            title VARCHAR,
            content VARCHAR
        )
        """)
        
        # 插入示例数据
        conn.execute("""
        INSERT INTO chinese_documents VALUES
            ('doc1', '人工智能简介', '人工智能是计算机科学的一个分支，致力于开发能够模拟人类智能的系统。'),
            ('doc2', '机器学习基础', '机器学习是人工智能的一个子领域，专注于让计算机从数据中学习。'),
            ('doc3', '深度学习技术', '深度学习是机器学习的一种方法，使用多层神经网络进行复杂模式识别。'),
            ('doc4', '自然语言处理', '自然语言处理是人工智能的一个分支，研究计算机与人类语言的交互。')
        """)
        print("中文文档表已创建并填充数据")
        
        # 创建预处理后的中文文档表
        conn.execute("""
        CREATE TABLE preprocessed_chinese_docs (
            doc_id VARCHAR,
            title_processed VARCHAR,
            content_processed VARCHAR
        )
        """)
        
        # 获取原始数据并进行预处理
        documents = conn.execute("SELECT * FROM chinese_documents").fetchall()
        for doc in documents:
            doc_id, title, content = doc
            # 对标题和内容进行分词处理
            title_processed = preprocess_chinese_text(title)
            content_processed = preprocess_chinese_text(content)
            
            # 将处理后的文本插入到新表中
            conn.execute("""
            INSERT INTO preprocessed_chinese_docs VALUES (?, ?, ?)
            """, (doc_id, title_processed, content_processed))
            
        print("中文文本已成功预处理")
        
        # 检查预处理后的表数据
        processed_docs = conn.execute("SELECT * FROM preprocessed_chinese_docs").fetchall()
        print("\n预处理后的文档示例：")
        for doc in processed_docs[:2]:  # 只显示前两个
            print(f"ID: {doc[0]}")
            print(f"处理后标题: {doc[1]}")
            print(f"处理后内容: {doc[2]}")
            print("-" * 50)
        
        # 为预处理后的中文文档创建FTS索引
        conn.execute("""
        PRAGMA create_fts_index(
            'preprocessed_chinese_docs', 'doc_id', 'title_processed', 'content_processed'
        )
        """)
        print("已为预处理后的中文文档创建FTS索引")
        
        # 定义搜索函数
        def search_chinese(query_text):
            # 对查询文本进行分词
            processed_query = preprocess_chinese_text(query_text)
            print(f"原始查询: '{query_text}'")
            print(f"处理后的查询: '{processed_query}'")
            
            # 执行搜索
            results = conn.execute("""
            SELECT sq.doc_id, cd.title, cd.content, score
            FROM (
                SELECT *, fts_main_preprocessed_chinese_docs.match_bm25(
                    doc_id, 
                    ?,  -- 使用处理后的查询字符串
                    fields := 'title_processed, content_processed'
                ) AS score
                FROM preprocessed_chinese_docs
            ) sq
            JOIN chinese_documents cd ON sq.doc_id = cd.doc_id
            WHERE score IS NOT NULL
            ORDER BY score DESC
            """, (processed_query,)).fetchall()
            
            return results
        
        # 执行搜索示例
        print("\n搜索示例1：")
        results1 = search_chinese("机器学习和人工智能")
        if results1:
            for result in results1:
                doc_id, title, content, score = result
                print(f"文档ID: {doc_id}, 标题: {title}")
                print(f"相关性得分: {score:.6f}")
        else:
            print("未找到匹配结果")
        
        print("\n搜索示例2：")
        results2 = search_chinese("神经网络")
        if results2:
            for result in results2:
                doc_id, title, content, score = result
                print(f"文档ID: {doc_id}, 标题: {title}")
                print(f"相关性得分: {score:.6f}")
        else:
            print("未找到匹配结果")
            
    finally:
        # 清理资源，关闭数据库连接
        conn.close()
        print("\nDuckDB连接已关闭")

if __name__ == "__main__":
    run_chinese_fts_demo()  # 执行中文FTS演示程序

FTS扩展已加载
中文文档表已创建并填充数据
中文文本已成功预处理

预处理后的文档示例：
ID: doc1
处理后标题: 人工智能 简介
处理后内容: 人工智能 计算机科学 分支 致力于 开发 能够 模拟 人类 智能 系统
--------------------------------------------------
ID: doc2
处理后标题: 机器 学习 基础
处理后内容: 机器 学习 人工智能 子 领域 专注 让 计算机 数据 中 学习
--------------------------------------------------
已为预处理后的中文文档创建FTS索引

搜索示例1：
原始查询: '机器学习和人工智能'
处理后的查询: '机器 学习 人工智能'
文档ID: doc2, 标题: 机器学习基础
相关性得分: 0.578844
文档ID: doc3, 标题: 深度学习技术
相关性得分: 0.289422
文档ID: doc1, 标题: 人工智能简介
相关性得分: 0.158721

搜索示例2：
原始查询: '神经网络'
处理后的查询: '神经网络'
未找到匹配结果

DuckDB连接已关闭
